In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '5'

import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
np.random.seed(7)

print('Python version : ', sys.version)
print('TensorFlow version : ', tf.__version__)
print('Keras version : ', keras.__version__)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
import numpy as np
np.random.seed(7)

print('Python version : ', sys.version)
print('TensorFlow version : ', tf.__version__)
print('Keras version : ', keras.__version__)

img_rows = 28
img_cols = 28

(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

input_shape = (img_rows, img_cols, 1)
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.



batch_size = 128
num_classes = 10
epochs = 12
index = np.random.permutation(x_train.shape[0])
new_m = 500
unlabeled_m = 10000
x_train2 = x_train[index][new_m:new_m+unlabeled_m]
y_train2 = y_train[index][new_m:new_m+unlabeled_m]
x_train = x_train[index][:new_m]
y_train = y_train[index][:new_m]


print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

y_train = keras.utils.to_categorical(y_train, num_classes)
y_train2 = keras.utils.to_categorical(y_train2, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

Using TensorFlow backend.


Python version :  3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]
TensorFlow version :  1.5.1
Keras version :  2.2.2
Python version :  3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]
TensorFlow version :  1.5.1
Keras version :  2.2.2
x_train shape: (500, 28, 28, 1)
y_train shape: (500,)
500 train samples
10000 test samples


In [2]:
def CreateCNN():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1), padding='same',
                     activation='relu',
                     input_shape=input_shape))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (2, 2), activation='relu', padding='same'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    
    #model.summary()
    return model

model = CreateCNN()
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Phase 1 training
hist = model.fit(x_train, y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 verbose=1, 
                 validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)

print('Phase1 test loss:', score[0])
print('Phase1 test accuracy:', score[1])
accPhase1 = score[1]

Train on 500 samples, validate on 10000 samples
Epoch 1/12
500/500 [==============================] - 8s 15ms/step - loss: 2.1950 - acc: 0.1840 - val_loss: 1.9317 - val_acc: 0.4181
Epoch 2/12
500/500 [==============================] - 1s 2ms/step - loss: 1.6986 - acc: 0.5400 - val_loss: 1.2173 - val_acc: 0.7485
Epoch 3/12
500/500 [==============================] - 1s 2ms/step - loss: 1.1078 - acc: 0.7020 - val_loss: 0.7420 - val_acc: 0.8055
Epoch 4/12
500/500 [==============================] - 1s 1ms/step - loss: 0.8040 - acc: 0.7040 - val_loss: 0.5782 - val_acc: 0.8230
Epoch 5/12
500/500 [==============================] - 1s 1ms/step - loss: 0.6191 - acc: 0.7960 - val_loss: 0.5021 - val_acc: 0.8386
Epoch 6/12
500/500 [==============================] - 1s 1ms/step - loss: 0.4535 - acc: 0.8360 - val_loss: 0.4406 - val_acc: 0.8630
Epoch 7/12
500/500 [==============================] - 1s 2ms/step - loss: 0.3559 - acc: 0.8740 - val_loss: 0.4169 - val_acc: 0.8648
Epoch 8/12
500/500 [=======

In [ ]:
# Phas2 training

for i in range(100):
    pseudo_label = model.predict_proba(x_train2).astype(np.uint64)
    x_train3 = np.vstack([x_train, x_train2])
    y_train3 = np.vstack([y_train, pseudo_label])
    index = np.random.permutation(x_train3.shape[0])
    x_train3 = x_train3[index]
    y_train3 = y_train3[index]

    hist = model.fit(x_train3, y_train3,
                     batch_size=batch_size,
                     epochs=epochs,
                     verbose=0, 
                     validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Trial ', i + 1, ' phase2 accuracy:', score[1] , ' (phase1', accPhase1, ')')
    
    

Trial  1  phase2 accuracy: 0.9086  (phase1 0.9105 )
Trial  2  phase2 accuracy: 0.9492  (phase1 0.9105 )
Trial  3  phase2 accuracy: 0.9477  (phase1 0.9105 )
Trial  4  phase2 accuracy: 0.9475  (phase1 0.9105 )
Trial  5  phase2 accuracy: 0.957  (phase1 0.9105 )
Trial  6  phase2 accuracy: 0.9532  (phase1 0.9105 )
Trial  7  phase2 accuracy: 0.9573  (phase1 0.9105 )
Trial  8  phase2 accuracy: 0.961  (phase1 0.9105 )
Trial  9  phase2 accuracy: 0.9675  (phase1 0.9105 )
Trial  10  phase2 accuracy: 0.97  (phase1 0.9105 )
Trial  11  phase2 accuracy: 0.9654  (phase1 0.9105 )
Trial  12  phase2 accuracy: 0.9695  (phase1 0.9105 )
Trial  13  phase2 accuracy: 0.9706  (phase1 0.9105 )
Trial  14  phase2 accuracy: 0.9712  (phase1 0.9105 )
Trial  15  phase2 accuracy: 0.973  (phase1 0.9105 )
Trial  16  phase2 accuracy: 0.9725  (phase1 0.9105 )
Trial  17  phase2 accuracy: 0.9735  (phase1 0.9105 )
Trial  18  phase2 accuracy: 0.9757  (phase1 0.9105 )
Trial  19  phase2 accuracy: 0.9766  (phase1 0.9105 )


In [ ]:
# Final evaluation accuracy

score = model.evaluate(x_test, y_test, verbose=0)
print('Final evaluation accuracy:', score[1] , ' from ', accPhase1)